In [1]:
# dependencies
"""
Numpy: matrix manipulation and math
Pandas: csv parsing and various data structure tasks
Mathpltlib.pyplot: data visualization
set_trace: debug breaks
keras: a machine learning library that is intuitive to read
tensorflow: backend for keras... also the most widely used machine learning library
re: regular expressions
"""
from copy import deepcopy as copy
from IPython.core.debugger import set_trace

import numpy as np
import pandas as pd
import scipy.special as sci
import matplotlib.pyplot as plt 
import os
import tensorflow as tf
import keras
# not needed
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

tf.config.optimizer.set_jit(True) # optimizes cpu usage

import re
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

Using TensorFlow backend.


In [2]:
"""
concat_files
----------

Concatenate text files in a directory as a string

dependent on 'os' Python module

parameters
----------
directory: string; path of the target directory

f_type:    tuple of strings; target file extentsions Ex: ('.py', '.cpp')

return
---------
content:   concatenated string

"""
def concat_files(directory,f_type):
    import os
    # List all file in the dataset directory
    # ------------------
    all_file = []
    content = ""

    # walk through every directory and open every f_type file
    # concatenate into var string "content"
    for root, dirs, files in os.walk(directory): 
        for name in files:
            if name.endswith(f_type): # we only care about .py
                all_file.append(name)
                with open(os.path.join(root,name), "r") as f:
                    content += f.read() + "\n"
    return content

In [3]:
content = concat_files("dataset",('.py'))

In [4]:
r_all_ascii = "[^\x00-\x7F]"

In [5]:
"""
encode_string
-----------
Generate a dictionary representation of the characters found 
in a string keyed with integer representations

Returns two dictionaries and an array. The two dictionaries are 
necessary to convert the string to integer representation
and back again. The array is the string encoded as integer values.

parameters
----------
content:      string; to be processed

return
----------
vocab_to_int: dict; character to integer representation of unique characters in the string

int_to_vocab: dict; integer to string representation

encoded:      array; string encoded as integer values
"""

def encode_string(content):   
    # Convert the string "content" into a list of intergers
    
    ### creates a set of the individual characters
    vocab = set(content)
    ### attempt to clean out non-ascii characters
    vocab_c = copy(vocab)
    for i, char in enumerate(vocab_c):
        if re.search(r_all_ascii,char):
            vocab.remove(char)
    print(vocab)
    print(len(vocab))
    ### use the set to sequentially generate a dictionary
    vocab_to_int = {c: i for i, c in enumerate(vocab)} 
    print(vocab_to_int)
    ### make keys the numerical values
    int_to_vocab = dict(enumerate(vocab)) 
    
    ### encode the "content" string using dict
    ### encoded = np.array([vocab_to_int[c] for c in content], dtype=np.int32)
    
    # *** Uncomment the below lines if you haven't saved the encoded array
    # Then rerun cell
#   -------------------------------------------------
#     encoded = np.array([],dtype=np.int16)
#     for c in content:
#         if c in vocab_to_int:
#             encoded = np.append(encoded,vocab_to_int[c])
#   -------------------------------------------------
    encoded = np.load('./encoded.npy') # comment out if above lines are uncommented
    
    return vocab_to_int, int_to_vocab, encoded

In [6]:
vocab_to_int, int_to_vocab, encoded = encode_string(content)

{'I', 'V', ':', '8', '^', 'm', 'a', 'h', 'L', 'Z', '-', 'l', 'i', 'W', 'k', 'q', '7', '|', ' ', '{', 'B', 'S', '*', 'C', ']', "'", '@', ';', 'v', '5', '>', 'F', '&', 'o', 'O', 'K', 'E', 's', 'p', '(', '\t', '~', 'J', 'n', '!', '3', 'c', '$', 'y', '4', '\n', '.', '`', ',', '=', '6', 'R', 'e', 'g', '"', ')', 'w', '\\', 'U', 'j', '+', 'u', '9', 'G', 'X', 'N', 'f', '<', 'Y', 'T', '[', 'P', 't', '?', 'z', 'Q', 'H', 'A', '1', 'M', '_', '0', 'x', '%', 'D', 'b', '2', '#', 'd', '/', '}', 'r'}
97
{'I': 0, 'V': 1, ':': 2, '8': 3, '^': 4, 'm': 5, 'a': 6, 'h': 7, 'L': 8, 'Z': 9, '-': 10, 'l': 11, 'i': 12, 'W': 13, 'k': 14, 'q': 15, '7': 16, '|': 17, ' ': 18, '{': 19, 'B': 20, 'S': 21, '*': 22, 'C': 23, ']': 24, "'": 25, '@': 26, ';': 27, 'v': 28, '5': 29, '>': 30, 'F': 31, '&': 32, 'o': 33, 'O': 34, 'K': 35, 'E': 36, 's': 37, 'p': 38, '(': 39, '\t': 40, '~': 41, 'J': 42, 'n': 43, '!': 44, '3': 45, 'c': 46, '$': 47, 'y': 48, '4': 49, '\n': 50, '.': 51, '`': 52, ',': 53, '=': 54, '6': 55, 'R': 56, 'e

## $\rightarrow$ Save encoded array to avoid heavy computation

In [7]:
#from tempfile import TemporaryFile as TF
outfile = "./encoded"

np.save(outfile,encoded)

In [8]:
#print(content)
print(int_to_vocab)
# this is all of the files concatenated. with each character encoded using the int_to_vocab
print(encoded)

{0: 'I', 1: 'V', 2: ':', 3: '8', 4: '^', 5: 'm', 6: 'a', 7: 'h', 8: 'L', 9: 'Z', 10: '-', 11: 'l', 12: 'i', 13: 'W', 14: 'k', 15: 'q', 16: '7', 17: '|', 18: ' ', 19: '{', 20: 'B', 21: 'S', 22: '*', 23: 'C', 24: ']', 25: "'", 26: '@', 27: ';', 28: 'v', 29: '5', 30: '>', 31: 'F', 32: '&', 33: 'o', 34: 'O', 35: 'K', 36: 'E', 37: 's', 38: 'p', 39: '(', 40: '\t', 41: '~', 42: 'J', 43: 'n', 44: '!', 45: '3', 46: 'c', 47: '$', 48: 'y', 49: '4', 50: '\n', 51: '.', 52: '`', 53: ',', 54: '=', 55: '6', 56: 'R', 57: 'e', 58: 'g', 59: '"', 60: ')', 61: 'w', 62: '\\', 63: 'U', 64: 'j', 65: '+', 66: 'u', 67: '9', 68: 'G', 69: 'X', 70: 'N', 71: 'f', 72: '<', 73: 'Y', 74: 'T', 75: '[', 76: 'P', 77: 't', 78: '?', 79: 'z', 80: 'Q', 81: 'H', 82: 'A', 83: '1', 84: 'M', 85: '_', 86: '0', 87: 'x', 88: '%', 89: 'D', 90: 'b', 91: '2', 92: '#', 93: 'd', 94: '/', 95: '}', 96: 'r'}
[48 59 31 ... 55 56 56]


In [25]:
segment = [300,600,900,1200]
# for i in range(500): 
#     segment.append((int(len(encoded) / 500) * (i + 1)))
batch = {
    "x" : [
        encoded[:segment[0]],
        encoded[segment[0]:segment[1]]
    ],
    "y" : [
        encoded[1:segment[0]+1],
        encoded[segment[0]+1:segment[1]+1]
    ]
}

In [28]:
# inp = np.vstack((batch['x'][0],batch['x'][1]))
# targets = np.vstack((batch['y'][0],batch['y'][1]))
inp = batch['x'][0]
targets = batch['y'][0]
Xtest = batch['x'][1]
Ttest = batch['y'][1]
io_size = len(int_to_vocab)

# inp = np.hstack((np.ones((Xtest.shape[0],1)),tf.one_hot(inp,io_size)))
# targets = np.hstack((np.ones((Xtest.shape[0],1)),tf.one_hot(targets,io_size)))
# Xtest = np.hstack((np.ones((Xtest.shape[0],1)),tf.one_hot(Xtest,io_size)))
# Ttest = np.hstack((np.ones((Xtest.shape[0],1)),tf.one_hot(Ttest,io_size)))

inp = tf.one_hot(inp,io_size)
targets = tf.one_hot(targets,io_size)
Xtest = tf.one_hot(Xtest,io_size)
Ttest = tf.one_hot(Ttest,io_size)

# encoded_as_i = np.hstack((np.ones((1,1)),tf.one_hot(encoded[:1],io_size)))
encoded_as_i = tf.one_hot(encoded[:40],io_size)

#tf.print(Xtest, summarize=50)
print(inp.shape)

# inp = tf.expand_dims(inp,2)
# targets = tf.expand_dims(targets,2)

(300, 97)


In [109]:
# lines = []
# temp = []
# for i, char in enumerate(encoded):
#     if int_to_vocab[char] == '\n':
#         lines.append(temp)
#     else:
#         temp.append(char)
#         temp = []

# x = np.zeros((len(lines), maxlen, len(int_to_vocab)), dtype=np.bool)
# y = np.zeros((len(sentences), len(int_to_vocab)), dtype=np.bool)
# for i, sentence in enumerate(sentences):
#     for t, char in enumerate(sentence):
#         x[i, t, char_indices[char]] = 1
#     y[i, char_indices[next_chars[i]]] = 1

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.optimizers import RMSprop, SGD

# x_train = np.hstack((batch['x'][0],batch['x'][1]))
# y_train = np.hstack((batch['y'][0],batch['y'][1]))
# x_test = np.hstack((batch['x'][0],batch['x'][1]))
# y_test = np.hstack((batch['y'][0],batch['y'][1]))

max_features = 97
#maxlen = 40

model = Sequential()
model.add(Embedding(max_features, output_dim=97))
model.add(LSTM(97, input_shape=inp.shape))
model.add(Dropout(0.5))
model.add(Dense(io_size, activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.fit(Xtest, Ttest, steps_per_epoch=97, epochs=5)
#model.get_weights().shape

/home/bran/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
97/97 [==============================] - 93s 964ms/step - loss: 3.3607 - accuracy: 0.1746
Epoch 2/5
97/97 [==============================] - 92s 948ms/step - loss: 3.0469 - accuracy: 0.1955
Epoch 3/5
97/97 [==============================] - 93s 959ms/step - loss: 3.2144 - accuracy: 0.1953
Epoch 4/5
97/97 [==============================] - 93s 963ms/step - loss: 3.0073 - accuracy: 0.2015
Epoch 5/5
97/97 [==============================] - 93s 958ms/step - loss: 2.6702 - accuracy: 0.2258


In [33]:
from keras.models import load_model
model.save('./model')
score = model.evaluate(inp, targets, steps=97)
print(score)

97/97 [==============================] - 26s 270ms/step
[0.04106758304477967, 15.519999653100967]


In [64]:
# print(tf.keras.backend.shape(Xtest),tf.keras.backend.shape(model.weights))
# sci.softmax(Xtest @ encoded_to_i)
print(encoded_as_i)
print(tf.convert_to_tensor(model.predict(encoded_as_i, steps=3, verbose=0)))
print(np.argmax(model.predict(encoded_as_i, steps=3, verbose=0),axis=0))
print(np.sum(model.predict(encoded_as_i, steps=3, verbose=0)[0]))
nextc = int_to_vocab[np.argmax(model.predict(encoded_as_i, steps=3, verbose=0),axis=0)[1]]
print(nextc) # should give probabilities of next character
print()

tf.Tensor(
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(40, 97), dtype=float32)
tf.Tensor(
[[4.0382799e-02 3.6107301e-07 2.2326464e-07 ... 2.5392863e-05
  2.9392481e-07 2.9682593e-07]
 [3.0115703e-02 9.5992255e-07 5.8833479e-07 ... 3.8130096e-05
  7.7778634e-07 7.9772832e-07]
 [4.4122519e-04 1.1850213e-07 7.9293002e-08 ... 4.3012276e-02
  1.0135348e-07 1.2478394e-07]
 ...
 [1.2823937e-03 1.7435547e-06 1.2205473e-06 ... 4.7490899e-05
  1.3142378e-06 1.4809921e-06]
 [1.5982764e-02 4.5820761e-06 2.8151956e-06 ... 7.4550655e-05
  3.8831436e-06 4.0619348e-06]
 [3.8662888e-02 4.1631239e-07 2.5747269e-07 ... 2.6746873e-05
  3.3779594e-07 3.4244664e-07]], shape=(120, 97), dtype=float32)
[ 0  8  8 34  8  8  0  0 34  8  8  8 34 34 34  8 34  8 34 34  0  0  0  1
  1 34  8  8  8  9  8  8 34  8  8  8  8  8  8  8  8 34  8 34 34  8 34  8
 34  8 34 13 34  8 11 34  8  8 34  8  8 34  8  

In [35]:
strout = ""
data = Xtest
#actual = ""
predict = np.argmax(model.predict(data, steps=97, verbose=0),axis=0)
print(predict)
for i in predict:
    
    nextc = int_to_vocab[predict[i]]
    
    #actual += int_to_vocab[data[i]]
    strout += nextc
print(strout)
#print(actual)


[ 30  24  30  30  30  30  61   0  30  30  61   1  30  30  30  30   8  14
  34  30  30  30  30   0   1  42  14  42  33   9  30   3  30  30   2   9
   8   0  30  30  30  30  30 217  87  87  30   2  30  30 205  30  30  20
  43 122   5  14  30  30  30  30  30 121   9 281  61 274  30 280  30  30
  30 121  30  64  30  30  30   6  30  30   0  30  31  30  30  13  30   8
 156  61  30 274  30  30  41]


IndexError: index 217 is out of bounds for axis 0 with size 97

# References

1. [LSTM: A search space odyssey](https://arxiv.org/pdf/1503.04069.pdf?utm_content=buffereddc5&utm_medium=social&utm_source=plus.google.com&utm_campaign=buffer)